In [3]:
from construct import *

data = None
tiff_file = 'data/data.NEF'
with open(tiff_file, 'rb') as f:
    data = f.read()


# TIFF

参考标准 [TIFF6.0](https://web.archive.org/web/20180810205359/https://www.adobe.io/content/udp/en/open/standards/TIFF/_jcr_content/contentbody/download/file.res/TIFF6.pdf)

# TIFF Struct

TIFF 文件最大为 2**32 bytes, 也就是 4GB.

TIFF 由一个 **Image File Header 和若干个 **Image File Directory(IFD)** 以及实际的数据组成

结构图如下:

![图 1](../asset/bf5757e490b74e3036f7972526d2a99880ec61076cf0aa2a13c01d45854089ac.png)  

## Image File Header

头部信息一共 8 个字节

- Bytes 0-1: 指示大小端
    - II(0x49 0x49): 小端字节序
    - MM(0x4D 0x4D): 大端字节序
- Bytes 2-3: 固定值 42, 进一步标识该文件为 TIFF 文件
    - 小端序: 0x2A 0x00
    - 大端序: 0x00 0x2A
- Bytes 4-7: 指向第一个 IFD 的 offset, 第一个 IFD 可能存在于 Image File Header 之后的任意地方, 但其偏移必须是偶数

![图 2](../asset/4371d66ff58c949220a99dd41aa537f9404921f1914bd995d0ecb6f6bab07a43.png)  


## Image File Directory(IFD)

每个 TIFF 文件必须要有至少一个 IFD, 而每个 IFD 至少有一个 IFD Entry 

IFD的结构为:

- Bytes 0-1: 指示 IFD 中包含 IFD Entry 的数量, 这里假设为 B
- 随后的 12 * B 个字节: 这里包含了 B 个 DE, 每个 DE 大小为 12 Bytes
- 随后的 4 个字节: 下一个 IFD 的偏移量, 如果为 0, 则表示当前为最后一个 IFD 

![图 1](../asset/bf5757e490b74e3036f7972526d2a99880ec61076cf0aa2a13c01d45854089ac.png)  

### IFD Entry

每个 IFD Entry 为 12 Bytes

结构如下:
- Bytes 0-1: Tag
- Bytes 2-3: Type
- Bytes 4-7: Value 的数量
- Bytes 8-11: Value 的值 或 Value 的起始 offset, 取决于 Type, offset 可以在文件的任何地方, offset 必须是偶数

![图 3](../asset/2e76318ec5802dd5faeebad564cce5fd2b35b16e95b76ae3b2cb3f7c8b3029ce.png)  


In [15]:
class TIFF:
    def __init__(self, data):
        self.data = data
        self.is_little = data[:2] == b'II'
        self.header_fmt = Struct(
            "ByteOrder" / Bytes(2),
            "MagicNum" / BytesInteger(2, swapped=self.is_little),
            "FirstIFDOffset" / BytesInteger(4, swapped=self.is_little),
        )

        self.IFD_entry_fmt = Struct(
            "Tag" / BytesInteger(2, swapped=self.is_little),
            "Type" / BytesInteger(2, swapped=self.is_little),
            "Count" / BytesInteger(4, swapped=self.is_little),
            "Data" / BytesInteger(4, swapped=self.is_little),
        )

        self.IFD_fmt = Struct(
            "IFDEntryCount" / BytesInteger(2, swapped=self.is_little),
            "IFDEntries" / Array(this.IFDEntryCount, self.IFD_entry_fmt),
            "NextIFDOffset" / BytesInteger(4, swapped=self.is_little)
        )

        self.header = self.header_fmt.parse(self.data)
        self.IFD = self.IFD_fmt.parse(data[self.header.FirstIFDOffset:])
    
    def __str__(self):
        ret = f'Header {self.header.__str__()}\n'
        ret += f'IFD {self.IFD.__str__()}\n'
        return ret

tiff = TIFF(data)
print(tiff)

Header Container: 
    ByteOrder = b'II' (total 2)
    MagicNum = 42
    FirstIFDOffset = 8
IFD Container: 
    IFDEntryCount = 25
    IFDEntries = ListContainer: 
        Container: 
            Tag = 254
            Type = 4
            Count = 1
            Data = 1
        Container: 
            Tag = 256
            Type = 4
            Count = 1
            Data = 160
        Container: 
            Tag = 257
            Type = 4
            Count = 1
            Data = 120
        Container: 
            Tag = 258
            Type = 3
            Count = 3
            Data = 316
        Container: 
            Tag = 259
            Type = 3
            Count = 1
            Data = 1
        Container: 
            Tag = 262
            Type = 3
            Count = 1
            Data = 2
        Container: 
            Tag = 271
            Type = 2
            Count = 18
            Data = 324
        Container: 
            Tag = 272
            Type = 2
            Count = 11